In [1]:
import sys
sys.path.append("../.")
import torch
import torchaudio
import numpy as np
import IPython.display as ipd
from model import WaveCRN
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
output_notebook()

No CUDA runtime is found, using CUDA_HOME='/usr'
/home/ubuntu/anaconda3/envs/WaveCRN/lib/python3.10/site-packages/torch/utils/cpp_extension.py:2356: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(
/home/ubuntu/anaconda3/envs/WaveCRN/lib/python3.10/site-packages/torch/cuda/__init__.py:789: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/home/ubuntu/anaconda3/envs/WaveCRN/lib/python3.10/site-packages/sru/cuda_functional.py:23: UserWarning: Just-in-time loading and compiling the CUDA kernels of SRU was unsuccessful. Got the following error:
list index out of range
  warnings.warn("Just-in-time loading and compiling the CUDA kernels of SRU was unsuccessful. "


Loading BokehJS ...

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# device = 'cpu'
net = WaveCRN()
model = torch.load("weights.pt", map_location="cpu")
net.to(device)
net.eval()

WaveCRN(
  (net): ConvBSRU(
    (conv): Conv1d(1, 256, kernel_size=(96,), stride=(48,), padding=(48,), bias=False)
    (deconv): ConvTranspose1d(256, 1, kernel_size=(96,), stride=(48,), padding=(48,), bias=False)
    (outfc): Linear(in_features=512, out_features=256, bias=False)
    (sru): SRU(
      (rnn_lst): ModuleList(
        (0): SRUCell(256, 256, rnn_dropout=0.1, bidirectional=True, rescale=False, layer_norm=True)
        (1-5): 5 x SRUCell(512, 256, rnn_dropout=0.1, bidirectional=True, rescale=False, layer_norm=True)
      )
    )
  )
)

In [14]:
import librosa as lb

noisy, sr = lb.load('noisy.wav', sr = 16000)
clean, sr = lb.load('clean.wav', sr = 16000)

noisy = torch.from_numpy(noisy).unsqueeze(0)
clean = torch.from_numpy(clean).unsqueeze(0)

pad_length = 48 - noisy.size(-1) % 48
noisy_padded = torch.nn.functional.pad(noisy, (0, pad_length))
noisy_padded.unsqueeze_(0)

tensor([[[0.0130, 0.0229, 0.0206,  ..., 0.0000, 0.0000, 0.0000]]])

In [15]:
with torch.no_grad():
    enhanced = net(noisy_padded.to(device))
enhanced = enhanced[0, :, :-pad_length]

In [21]:
import IPython.display as idp 

idp.Audio(enhanced, rate = 16000)

In [18]:
p = figure(width=800, height=400)
x_axis = np.linspace(0, clean.size(-1) / sr, num=clean.size(-1))
p.multi_line(
    [x_axis, x_axis],
    [enhanced[0].cpu().numpy(), clean[0].cpu().numpy()],
    color=['blue', 'green'], 
    line_width=1
)

show(p)